In [1]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import os
import pandas as pd
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler

In [2]:
%run common.ipynb

In [3]:
%time
pickle_df = pd.read_pickle('train.pkl')

print(pickle_df.shape, type(pickle_df))
print(pickle_df[15:20])

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.78 µs
(312217, 3) <class 'pandas.core.frame.DataFrame'>
       ari_time                                          ari_title  price
119  2010-06-13  Bank of England Faces Debate as Inflation Show...      1
150  2010-06-13  Besix Wins $265 Million Contract to Build Bahr...      1
164  2010-06-13  Brazil to Buy as Many Petrobras Shares as Poss...      1
72   2010-06-13  Cablevision Said Near Purchase of Bresnan Comm...      1
66   2010-06-13  Cameron Appoints Adviser on U.K. Safety Rules ...      1


In [4]:
# 读取之前生成的news资料里的标题
# 并加上句号以及分割成一行一句格式
# 以便 reverb 工具来读取
file_path = "./reverb_pre.txt"

file_num = 0
error_num = 0
with open(file_path, "w+", encoding='utf-8') as fo:
    for ari_title in pickle_df[:-1]["ari_title"]:
        file_num+=1
        try:
            fo.write(ari_title + ".\n")
        except Exception as e:
#             error_num+=1
#             error_str = u''.join((ari_title)).encode('utf-8')
#             print(str(e) ,error_str)
#             fo.write(ari_title + ".\n")
            continue 
print("\n\n error_num percentage: ", 100* error_num/file_num , " %")        
print ("Saved:", fo.name)



 error_num percentage:  0.0  %
Saved: ./reverb_pre.txt


In [5]:
# Call Reverb for OpenIE parsing
# Run command below before continuing on the next step:
#   java -Xmx512m -jar ./test-reverb/reverb.jar --quiet reverb_pre.txt > reverb_result.txt

In [6]:
%%time

# Create date-dataframe dict with all dates and empty title
date_df_dict = OrderedDict()
# [Checking] number of titles (processed by reverb) per day
date_titles_count = OrderedDict()
for ari_time, price in zip(pickle_df['ari_time'], pickle_df['price']):
    # Extract date & price
    date_t = ari_time[0:10]
    price_t = int(price)  # 1 or 0
    # Add dataframe with empty title
    date_df_dict[date_t] = [date_t, '', price_t]
    date_titles_count[date_t] = 0


# reverb tool 生成的结果文件,命令行生成的,带有额外杂讯
result_file_name = "reverb_result.txt"
file_content = ""
with open(result_file_name, 'r', encoding='utf-8') as out:
    file_content = out.read()
res_content_list = file_content.split("\n")

# Remove last line (empty string)
del res_content_list[-1]

# 提取 reverb 结果文件中需要的三元组信息
src_line_numbers = []
reverb_res_list = []
for fc in range(0, len(res_content_list)):
    line_tokens = res_content_list[fc].split("\t")
    src_line_numbers.append(int(line_tokens[1]) - 1)
    reverb_res_list.append(line_tokens[-3:])

# reverb 并没有把所有news title 全部生成三元组格式
date_title_list = []  # titles within same day
last_date = ''
i = 0
for terms in log_progress(reverb_res_list):
    terms = reverb_res_list[i]
    src_line_number = src_line_numbers[i]
    
    # Extract date & price
    date_t = pickle_df["ari_time"][src_line_number][0:10]
    price_t = date_df_dict[date_t][2]
    # Join terms as a string
    ari_title_t = ' '.join(terms)
    
    # Concat titles in the same day
    if last_date != date_t:
        if len(date_title_list) > 0:
            date_titles_count[last_date] = len(date_title_list)
            title_aggr = ' '.join(date_title_list)
            date_df_dict[last_date] = [last_date, title_aggr, price_t]
            date_title_list = []
        last_date = date_t
    
    date_title_list.append(ari_title_t)
    i += 1

print('[post-reverb] Number of days: %d' % len(date_titles_count))
print('[post-reverb] Titles per day:', date_titles_count)
# print(reverb_df_list)

del date_titles_count

[post-reverb] Number of days: 946
[post-reverb] Titles per day: OrderedDict([('2010-06-13', 6), ('2010-06-14', 8), ('2010-06-15', 4), ('2010-06-16', 34), ('2010-06-17', 24), ('2010-06-18', 67), ('2010-06-19', 4), ('2010-06-20', 9), ('2010-06-21', 3), ('2010-06-22', 20), ('2010-06-23', 1), ('2010-06-24', 29), ('2010-06-25', 42), ('2010-06-26', 29), ('2010-06-27', 6), ('2010-06-28', 6), ('2010-06-29', 4), ('2010-06-30', 25), ('2010-07-01', 2), ('2010-07-02', 2), ('2010-07-03', 3), ('2010-07-04', 9), ('2010-07-05', 1), ('2010-07-06', 11), ('2010-07-07', 18), ('2010-07-08', 19), ('2010-07-09', 8), ('2010-07-10', 14), ('2010-07-11', 3), ('2010-07-12', 46), ('2010-07-13', 3), ('2010-07-14', 34), ('2010-07-15', 8), ('2010-07-16', 32), ('2010-07-17', 8), ('2010-07-18', 1), ('2010-07-19', 31), ('2010-07-20', 7), ('2010-07-21', 86), ('2010-07-22', 10), ('2010-07-23', 129), ('2010-07-24', 11), ('2010-07-25', 4), ('2010-07-26', 2), ('2010-07-27', 20), ('2010-07-28', 46), ('2010-07-29', 33), ('2010

In [19]:
# 加入沒有被reverb 成功的其他title 資料
# Make sure news dataset has been extracted (follow instructions on README.md)
# pickle_df = pd.read_pickle('train.pkl')

date_df_dict_keys = list(date_df_dict.keys())
# print(date_df_dict_keys[:10])
index = 0
for now_date in date_df_dict_keys:
    title_temp = date_df_dict[now_date][1]
    if (title_temp == ""):
        title_aggr = ' '.join(pickle_df[index:(index+1)]["ari_title"])
        date_df_dict[now_date] = [now_date, title_aggr, 1]
    index+=1
           


In [20]:
# date_secs = [[utcdate2timestamp(df_row[0])] for df_row in reverb_df_list]

# for df_row, normalized_date in zip(reverb_df_list, date_secs):
#     print(normalized_date[0], "\n", df_row[0],"\n", df_row[1],"\n", df_row[2])
# #     print("\n", df_row,df_row[0], df_row[0],df_row[0],df_row[0]=="2010-11-19")
#     break
pickle_df[3:10]

,ari_time,ari_title,price
188,2010-06-13,"Air India, Boeing to Hold Talks Over Plane Del...",1
174,2010-06-13,"Al-Qaeda Gunmen Storm Iraqi Central Bank, Kill...",1
189,2010-06-13,"Algeria, Slovenia Tied 0-0 at Halftime of Worl...",1
185,2010-06-13,Asian Junk Bonds Risk Company Calls Amid Bigge...,1
129,2010-06-13,Audi Dominates Podium Positions at Le Mans 24-...,1
163,2010-06-13,Australia to Foot Bill for Rescue of U.S. Teen...,1
63,2010-06-13,Australia to Propose Tougher Penalties for Avi...,1


In [21]:
reverb_df_list = list(date_df_dict.values())

# Normalize date field
date_secs = [[utcdate2timestamp(df_row[0])] for df_row in reverb_df_list]

scaler = MinMaxScaler(feature_range=(0, 1))
date_secs = scaler.fit_transform(date_secs)

normalized_df_list = []
for df_row, normalized_date in zip(reverb_df_list, date_secs):
    normalized_df_list.append([df_row[0], df_row[1], df_row[2]])

# print(normalized_df_list)

In [22]:
# 真正要用来训练的资料
# 提取了三元组,并组合了对应的股票价格和时间
reverb_df = pd.DataFrame(normalized_df_list, columns=["ari_time", "ari_title", "price"])
print("Shape:", reverb_df.shape)
reverb_df.to_pickle('train_reduce.pkl')

Shape: (946, 3)


In [23]:
# [Checking] 输出训练资料查验
reverb_pkl_df = pd.read_pickle('train_reduce.pkl')
print("Shape: ", reverb_pkl_df.shape)
print( reverb_pkl_df[:20])

Shape:  (946, 3)
      ari_time                                          ari_title  price
0   2010-06-13  bank of new zealand say to start marketing nz$...      1
1   2010-06-14  banking regulator say forces driving china 's ...      1
2   2010-06-15  broad commits # % of wealth take buffett pledg...      0
3   2010-06-16  u.s. stocks trim losses south korea tell north...      1
4   2010-06-17  australian prosecutors get evidence from monta...      0
5   2010-06-18  reliance cede oil block reysas of turkey offer...      0
6   2010-06-19  kingsgate raise $ # # lebanon allow aid ships ...      0
7   2010-06-20  cez want to select its own fuel suppliers cadi...      1
8   2010-06-21  rate slow to # % canada 's harper say he he ke...      0
9   2010-06-22  million jury loss reinstate by appeals court a...      0
10  2010-06-23                   china stocks drop for second day      1
11  2010-06-24  speculation china allow appreciation administr...      0
12  2010-06-25  george williamson 